In [ ]:
import pandas as pd
import os.path

In [ ]:
weather_data_path = '/Users/nilsgamperli/Downloads/weather_data'
municipalities = pd.read_excel('../Data/municipalities/be-b-00.04-agv-20050313.xlsx', sheet_name='Gemeindeliste-Liste d. communes')

In [ ]:
# extract GDNAME
municipalities = municipalities['GDENAME']

In [ ]:
municipalities

In [ ]:
def checkMunicipalityWeatherData(data: pd.Series, path: str)->list:
    municipalitiesWithoutWeatherData = []
    for municipality in data:
        mun_path = path + '/' + municipality + '.json'
        if not os.path.isfile(mun_path):
            municipalitiesWithoutWeatherData.append(municipality)
    return municipalitiesWithoutWeatherData

In [ ]:
municipalitiesWithoutWeatherData = pd.Series(checkMunicipalityWeatherData(municipalities, weather_data_path))
municipalitiesWithoutWeatherData.to_csv('../Data/municipalities/municipalities_without_weather_data.csv')

### Now we need to check if the coordinates of those municipalities exist

In [ ]:
municipalities_coordinate = pd.read_csv('../Data/municipalities/municipality_coordinates.csv')
municipalities_coordinate = municipalities_coordinate

In [ ]:
municipalities_coordinate

In [ ]:
municipalitiesWithoutCoordinates = []
for municipality in municipalitiesWithoutWeatherData.tolist():
    if municipality not in municipalities_coordinate['Municipality'].tolist():
        municipalitiesWithoutCoordinates.append(municipality)

pd.Series(municipalitiesWithoutCoordinates).to_csv('../Data/municipalities/municipalities_without_coordinates.csv')
municipalitiesWithoutCoordinates

#### All Coordinates are present, get remaining weather data

In [ ]:
filtered_municipalities_coordinates = municipalities_coordinate[municipalities_coordinate['Municipality'].isin(municipalitiesWithoutWeatherData)]
filtered_municipalities_coordinates

In [ ]:
filtered_municipalities_coordinates.to_csv('/Users/nilsgamperli/PycharmProjects/StormMind/data_engineering/Data/municipalities/municipalities_to_rename.csv')

In [ ]:
import importlib
import data_engineering.snippets.requesting_weather_data as requests
importlib.reload(requests)
for i, row in filtered_municipalities_coordinates.iterrows():
    requests.fetch_and_save_json(row['Latitude'], row['Longitude'], row['Municipality'], '/Users/nilsgamperli/Downloads/weather_data')

 convert the names including a '/' in our main_data.csv

In [ ]:
import polars as pl

In [ ]:
dictionary = {
    'Biel/Bienne' : 'Biel',
    'Vaz/Obervaz' : 'Vaz',
    'Brienz/Brinzauls': 'Brienz',
    'Lantsch/Lenz': 'Lantsch',
    'Bergün/Bravuogn' : 'Bergün',
    'Waltensburg/Vuorz': 'Waltensburg',
    'Feldis/Veulden' : 'Feldis',
    'Tumegl/Tomils' : 'Tumegl',
    'Domat/Ems' : 'Domat',
    'Celerina/Schlarigna' : 'Celerina',
    'Sils im Engadin/Segl' : 'Sils im Engadin',
    'Breil/Brigels' : 'Breil',
    'Disentis/Mustér' : 'Disentis',
    'Bosco/Gurin' : 'Bosco'
}

In [ ]:
main_data = pl.read_csv('../Data/main_data.csv')

In [ ]:
main_data = main_data.with_columns([pl.all().cast(pl.Utf8).replace(dictionary)])

In [ ]:
main_data.filter(pl.col('Municipality') == 'Biel')

In [ ]:
main_data.write_csv('../Data/main_data.csv')

In [ ]:
## download the remaining weather data
rem_mun_coords = pd.read_csv('../Data/municipalities/municipalities_to_rename.csv')

In [ ]:
for i, row in rem_mun_coords.iterrows():
    requests.fetch_and_save_json(row['Latitude'], row['Longitude'], row['Municipality'], '/Users/nilsgamperli/Downloads/weather_data')